In [1]:
%run ./DevEnvironment.ipynb
%run ./DataLoaders.ipynb

Following locations can be used : 

`NOTEBOOKS_DIR`, `MANUSCRIPT_DIR`, `CODE_DIR`, `DATA_DIR`

Following locations can be used : 

`NOTEBOOKS_DIR`, `MANUSCRIPT_DIR`, `CODE_DIR`, `DATA_DIR`

Use `loadCSVtoDF` to load data into dataframes from folder: `SOME_DATA_DIR/raw/SOME_DATASET/SOME_MARKER/SOME_LAB` 

## Pandas settings

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_rows', 200)

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
adnifolder = DATA_DIR + 'ADNI/'
datafolder = adnifolder + 'raw/'
datasets = ['ADNIMERGE', 'CSF', 'PLASMA', 'PET', 'MRI', 'NEUROPSYCH', 'GDS']

## Load all datasets and dictionaries into an object of dataframes

In [5]:
debug = False
dataframes = {}
for dataset in datasets :
    folder = Path(datafolder + dataset + '/')
    for x in folder.glob('*') :
        loadCSVtoDF(folder, dataframes)
    
print('-----------------------')
print('Dataframes are available in the object - dataframes - with keys as [dataset], [marker], [file], [df | dict]')
print()
print('E.g : dataframes["PLASMA"]["ABETA"]["FNIH_SHIMADZU_1_GO_2_2021-05-25"]["df"] and ')
print('dataframes["PLASMA"]["ABETA"]["FNIH_SHIMADZU_1_GO_2_2021-05-25"]["dict"] and ')
print()
print('If the dataset is of not a specific measure, they are under the marker - ALL')
print('E.g : dataframes["CSF"]["ALL"]["LOCALLAB_1_GO_2_3_2022-01-24"]["df"] ')


-----------------------
Dataframes are available in the object - dataframes - with keys as [dataset], [marker], [file], [df | dict]

E.g : dataframes["PLASMA"]["ABETA"]["FNIH_SHIMADZU_1_GO_2_2021-05-25"]["df"] and 
dataframes["PLASMA"]["ABETA"]["FNIH_SHIMADZU_1_GO_2_2021-05-25"]["dict"] and 

If the dataset is of not a specific measure, they are under the marker - ALL
E.g : dataframes["CSF"]["ALL"]["LOCALLAB_1_GO_2_3_2022-01-24"]["df"] 


In [6]:
import copy
import researchpy as rp
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import cufflinks as cf
import statsmodels.api as sm
import statsmodels.formula.api as smf
from itertools import combinations
import seaborn as sns

import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

### Exporting dataframes to PPT
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE
from pptx.util import Pt, Cm, Inches
from pptx.enum.text import PP_ALIGN

from pd2ppt import df_to_powerpoint, df_to_table

import warnings
warnings.filterwarnings('ignore')

### ADNIMERGE

**ADNIMERGE might be straightforward**

**Check the exact file name of ADNIMERGE-ALL**

In [ ]:
if debug: 
    for f in dataframes["ADNIMERGE"]["ALL"]:
        print(f)

**Use the above name to create the ADNIMERGE dataframe**

In [ ]:
ADNIMERGE = dataframes["ADNIMERGE"]["ALL"]["ADNIMERGE_1_GO_2_3_2022-01-24"]["df"]

In [ ]:
def getRaceEthnicity(row) :
    if 'White' == row['PTRACCAT'] :
        if 'Hisp/Latino' == row['PTETHCAT'] :
            return 'Hispanic White'
        else :
            return 'Non-hispanic White'
    elif 'Black' == row['PTRACCAT'] :
        return 'Black or African American'
    else : return 'Other'
    
ADNIMERGE['ETHNICRACE'] = ADNIMERGE.apply(lambda row: getRaceEthnicity(row), axis=1)

In [ ]:
def monthFromViscode(viscode) :
    viscode = viscode.lower()
    if 'bl' in viscode:
        return 0
    elif viscode.startswith('m') :
        month = viscode[1:]
        return int(month)
    else :
        return -1

ADNIMERGE['VS_MONTH'] = ADNIMERGE.VISCODE.apply(lambda x: monthFromViscode(x))


In [ ]:
px.histogram(ADNIMERGE['VS_MONTH'])

In [6]:
ADNIMERGE.groupby(["RID", "VISCODE"])

NameError: name 'ADNIMERGE' is not defined

In [ ]:
def checkDuplicates(df, group_by_cols=["RID", "VISCODE"]) :
    dupes = df.groupby(group_by_cols)["RID"].count().reset_index(name='count')
    return dupes[dupes["count"] > 1]


In [ ]:
if False:
    print('--')
    print('dataframe for ADNIMERGE : ADNIMERGE')

    print('--')
    print('use checkDuplicates(df) to find out dupplicate rows for RID, VISCODE combination')

<a id="pet" />

### PET

**Available PET Measures**

In [ ]:
if debug:
    for k in dataframes["PET"] :
        print(k)

**PET Files with TAU measures**

* General goal :
    - A master dataset of PET TAU measure from all the available labs
* Main fields to be collected
    - RID, VISCODE (or VISCODE2)
    - 
    - Some timestamp indicating the sample collection time, depending on the lab

**Examine each lab file and the available fields**

In [ ]:
if debug:
    for k in dataframes["PET"]["TAU"] :
        print('#lab ="{}"'.format(k))
        #print(k)

**Check the dictionary where fields are unclear**

In [ ]:
if debug:
    marker = "PET"
    measure = "TAU"
    #lab ="BAI_NMRC_FAV1451_2_3_2020-03-03"
    lab ="UCB_AV1451_1_GO_2_3_2021-11-16"
    df = dataframes[marker][measure][lab]["df"]
    print(lab)
    print(df.columns)
    print(df.info())
    dict_unc = dataframes[marker][measure][lab]["dict"]
    dict_unc[["FLDNAME", "TEXT"]]
    #df.sample(5)

**Collect all PET_TAU datasets in a standardized format**

In [ ]:
pet_tau = {}

In [ ]:
pet_tau_columns = ["RID", "VISCODE", "PET_SUVR_TAU", "update_stamp", "source"]

**UCB_AV1451_1_GO_2_3_2021-11-16**
 - Composite SUVR values are given according to BRAAK regions - 1,2, 3/4,5/6
 - Braak Region 1 is Entorihnal Cortex
 - Braak Region 2 is Left and Right Hioppocampus but it has been mentioned in their methods that this is corrupted and not used in their analyses

In [ ]:
marker = "PET"
measure = "TAU"
lab ="UCB_AV1451_1_GO_2_3_2021-11-16"
df = dataframes[marker][measure][lab]["df"]
df = df[["RID", "VISCODE2", "BRAAK1_SUVR", "update_stamp"]]
df = df.dropna()
df["source"] = lab
pet_tau[lab] = pd.DataFrame(df)
pet_tau[lab].columns = pet_tau_columns

In [ ]:
if debug: pet_tau[lab].sample(5)

In [ ]:
if debug:
    print('--')
    print('dataframe dictionary for PET TAU {}: pet_tau')
    print('with columns : ', pet_tau_columns)

**PET Files with ABETA measures**

* General goal :
    - A master dataset of PET ABETA measure from all the available labs
* Main fields to be collected
    - RID, VISCODE (or VISCODE2)
    - AB42, AB40, AB42/40
    - Some timestamp indicating the sample collection time, depending on the lab

**Examine each lab file and the available fields**

In [ ]:
if debug:
    for k in dataframes["PET"]["ABETA"] :
        print('#lab ="{}"'.format(k))
        #print(k)

**Check the dictionary where fields are unclear**

In [ ]:
if debug:
    marker = "PET"
    measure = "ABETA"
    lab ="UCB_AV45_1_GO_2_3_2021-11-16"
    #lab ="BAI_NMRC_AV45_1_GO_2_3_2020-10-23"
    print(lab)
    df = dataframes[marker][measure][lab]["df"]
    print(df.columns)
    print(df.info())
    dict_unc = dataframes[marker][measure][lab]["dict"]
    dict_unc[["FLDNAME", "TEXT"]]
    #df.sample(5)

**Collect all PET_ABETA datasets in a standardized format**

In [ ]:
pet_abeta = {}

In [ ]:
pet_abeta_columns = ["RID", "VISCODE", "PET_SUVR_ABETA", "update_stamp", "source"]

**UCB_AV45_1_GO_2_3_2021-11-16**
 - Choosing the Composite SUVR value - COMPOSITE_SUVR

In [ ]:
marker = "PET"
measure = "ABETA"
lab ="UCB_AV45_1_GO_2_3_2021-11-16"
df = dataframes[marker][measure][lab]["df"]
# Refer to methods UCB AV45

#df = df[["RID", "VISCODE2", "COMPOSITE_SUVR", "update_stamp"]]
df = df[["RID", "VISCODE2", "SUMMARYSUVR_WHOLECEREBNORM", "update_stamp"]]

df = df.dropna()
df["source"] = lab
pet_abeta[lab] = pd.DataFrame(df)
pet_abeta[lab].columns = pet_abeta_columns

In [ ]:
# df = dataframes[marker][measure][lab]["df"]
# # Refer to methods UCB AV45
# print('COMPOSITE_SUVR > 1.11 (%): {0:.2f}'.format(100*(df[df["COMPOSITE_SUVR"] > 1.11].shape[0]/df.shape[0])))
# print('SUMMARYSUVR_WHOLECEREBNORM > 1.11 (%): {0:.2f}'.format(100*(df[df["SUMMARYSUVR_WHOLECEREBNORM"] > 1.11].shape[0]/df.shape[0])))
# print('SUMMARYSUVR_WHOLECEREBNORM > 1.11 (\%): {}'.format(100*(df[df["SUMMARYSUVR_WHOLECEREBNORM"] > 1.11].shape[0]/df.shape[0]))


In [ ]:
if debug: pet_abeta[lab].sample(5)

In [ ]:
if debug:
    print('--')
    print('dataframe dictionary for PET ABETA {}: pet_abeta')
    print('with columns : ', pet_abeta_columns)

<a id="mri" />

### MRI

**Available MRI Measures**

In [ ]:
for k in dataframes["MRI"]["ALL"]["UCSF_FS6_3_2021-12-13"] :
        print(k)
        
dataframes["MRI"]["ALL"]["UCSF_FS6_3_2021-12-13"]["dict"]["TEXT"];

* General goal :
    - Parking Free Surfer 4 data for now
    - Lots of fields with volumes
    - For now, sticking to RightHippocampus and LeftHippocampus
* Main fields to be collected
    - RID, VISCODE (or VISCODE2)
    - ST88SV - RightHippocampus 
    - ST29SV - LeftHippocampus
    - Some timestamp indicating the sample collection time, depending on the lab
* For all regions of interest, refer A4, normalise using Intra Cranial Volume (refer Ali's paper)

**Examine each lab file and the available fields**

In [ ]:
if debug :
    for k in dataframes["MRI"]["ALL"] :
        print('#lab ="{}"'.format(k))
        #print(k)

**Check the dictionary where fields are unclear**

In [ ]:
if debug:
    marker = "MRI"
    measure = "ALL"
    #lab ="UCSF_FS4_1_GO_2_2015-11-02"
    lab ="UCSF_FS5_1_GO_2_2021-12-13"
    #lab ="UCSF_FS6_3_2021-12-13"
    print(lab)
    df = dataframes[marker][measure][lab]["df"]
    print(df.columns)
    #print(df.info())
    dict_unc = dataframes[marker][measure][lab]["dict"]
    #dict_unc[["FLDNAME", "TEXT"]][~dict_unc["TEXT"].isna() & dict_unc["TEXT"].str.contains("(?i)Hipp")]
    dict_unc[["FLDNAME", "TEXT"]]
    #df.sample(5)

**Collect all MRI datasets in a standardized format**

In [ ]:
mri_hipp = {}

**The following are the fields of interest in terms of the volume of different brain regions**

In [ ]:
%run ./ADNI-MRI.ipynb

In [ ]:
regions = {}
ADD_HEMISPHERES = True
for region in sorted(MRI_REGIONS_VOLUME.keys()) :
    reg_ = region.upper()
    idx = 0
    side = "ALL"
    if ADD_HEMISPHERES :
        if reg_.startswith("LEFT") :
            idx = 4
            side = "LEFT"
        elif reg_.startswith("RIGHT") :
            idx = 5
            side = "RIGHT"
        reg_ = reg_[idx:]
    if reg_ not in regions : regions[reg_] = {}
    regions[reg_][side] = MRI_REGIONS_VOLUME[region]
if debug:
    for k in regions :
        print(k,  regions[k])


**For this analysis, we sum-up the volumes of Left and Right regions where applicable**

In [ ]:
mri_hipp_columns = ["RID", "VISCODE"] + ['VOL_'+k for k in sorted(regions.keys())] + ["update_stamp", "source"]

**UCSF_FS5_1_GO_2_2021-12-13**
 - Choosing the sub-cortical volumes of Left and Right Hippocampus

In [ ]:
marker = "MRI"
measure = "ALL"
lab ="UCSF_FS5_1_GO_2_2021-12-13"
df = dataframes[marker][measure][lab]["df"]

all_mri_fields = [regions[reg][side] for reg in regions.keys() for side in regions[reg] ]
mri_df = df[["RID", "VISCODE2"]]
#Add volumes of each side of each region and rename column names to region names
for reg_ in sorted(regions.keys()) : 
    if "ALL" in regions[reg_] : #like ICV
        mri_df['VOL_'+reg_] = df[regions[reg_]["ALL"]]  
    else : 
        mri_df['VOL_'+reg_] = df[regions[reg_]["LEFT"]] + df[regions[reg_]["RIGHT"]]
mri_df["update_stamp"] = df["update_stamp"]
# df = df.dropna()
# df["MRI_HIPP"] = df["ST88SV"] + df["ST29SV"]
mri_df["source"] = lab

In [ ]:

mri_hipp[lab] = pd.DataFrame(mri_df)
mri_hipp[lab].columns = mri_hipp_columns

In [ ]:
if debug: mri_hipp[lab].sample(5)

**UCSF_FS6_3_2021-12-13**
 - Choosing the sub-cortical volumes of Left and Right Hippocampus

In [ ]:
marker = "MRI"
measure = "ALL"
lab ="UCSF_FS6_3_2021-12-13"
df = dataframes[marker][measure][lab]["df"]
all_mri_fields = [regions[reg][side] for reg in regions.keys() for side in regions[reg] ]
mri_df = df[["RID", "VISCODE2"]]
#Add volumes of each side of each region and rename column names to region names
for reg_ in sorted(regions.keys()) : 
    if "ALL" in regions[reg_] : #like ICV
        mri_df['VOL_'+reg_] = df[regions[reg_]["ALL"]]  
    else : 
        mri_df['VOL_'+reg_] = df[regions[reg_]["LEFT"]] + df[regions[reg_]["RIGHT"]]
mri_df["update_stamp"] = df["update_stamp"]
# df = df.dropna()
# df["MRI_HIPP"] = df["ST88SV"] + df["ST29SV"]
mri_df["source"] = lab
mri_hipp[lab] = pd.DataFrame(mri_df)
mri_hipp[lab].columns = mri_hipp_columns

In [ ]:
if debug: mri_hipp[lab].sample(5)

In [ ]:
if debug:
    print('--')
    print('dataframe dictionary for MRI Hippocampus {}: mri_hipp')
    print('with columns : ', mri_hipp_columns)

<a id="csf" />

### CSF

**Available CSF Measures**

In [ ]:
if debug:
    for k in dataframes["CSF"] :
        print(k)

<a id="csf-abeta" />

**CSF Files with ABETA measures**

* General goal :
    - A master dataset of CSF ABETA measure from all the available labs
* Main fields to be collected
    - RID, VISCODE (or VISCODE2)
    - AB42, AB40, AB42/40
    - Some timestamp indicating the sample collection time, depending on the lab

**Examine each lab file and the available fields**

In [ ]:
if debug:
    for k in dataframes["CSF"]["ABETA"] :
        print('#lab ="{}"'.format(k))
        #print(k)

**Check the dictionary where fields are unclear**

In [ ]:
if debug:
    marker = "CSF"
    measure = "ABETA"
    lab ="FUJIREBIO_1_GO_2_2020-02-04"
    #lab ="EUROIMMUN_1_GO_2_2019-04-18"
    print(lab)
    df = dataframes[marker][measure][lab]["df"]
    print(df.columns)
    print(df.info())
    dict_unc = dataframes[marker][measure][lab]["dict"]
    dict_unc[["FLDNAME", "TEXT"]]
    df.sample(5)

**Collect all CSF_ABETA datasets in a standardized format**

In [ ]:
csf_abeta = {}

In [ ]:
csf_abeta_columns = ["RID", "VISCODE", "CSF_AB42", "CSF_AB40", "CSF_AB4240", "update_stamp", "source"]

**FUJIREBIO_1_GO_2_2020-02-04**
 - Looks standard
 - Fields of interest 
  - 'ABETA42'
  - 'ABETA40' 
  - 'ABETA42_40'

In [ ]:
marker = "CSF"
measure = "ABETA"
lab ="FUJIREBIO_1_GO_2_2020-02-04"
df = dataframes[marker][measure][lab]["df"]
df = df[["RID", "VISCODE2", "ABETA42", "ABETA40", "ABETA42_40", "update_stamp"]]
df = df.dropna()
df["source"] = lab
csf_abeta[lab] = pd.DataFrame(df)
csf_abeta[lab].columns = csf_abeta_columns

In [ ]:
if debug: csf_abeta[lab].sample(5)

**EUROIMMUN_1_GO_2_2019-04-18**
 - VISCODE2 **240** non-null but BETA_AMYLOID_42_40 **278 non-null** ?
 - Fields of interest 
  - 'BETA_AMYLOID_1_40'
  - 'BETA_AMYLOID_1_42'
  - 'BETA_AMYLOID_42_40'

In [ ]:
marker = "CSF"
measure = "ABETA"
lab ="EUROIMMUN_1_GO_2_2019-04-18"
df = dataframes[marker][measure][lab]["df"]
df = df[["RID", "VISCODE2", "BETA_AMYLOID_1_42", "BETA_AMYLOID_1_40", "BETA_AMYLOID_42_40", "update_stamp"]]
df = df.dropna()
df["source"] = lab
csf_abeta[lab] = pd.DataFrame(df)
csf_abeta[lab].columns = csf_abeta_columns

In [ ]:
if debug: csf_abeta[lab].sample(5)

In [ ]:
if debug :
    print('--')
    print('dataframe dictionary for CSF ABETA {}: csf_abeta')
    print('with columns : ', csf_abeta_columns)

<a id="csf-nfl" />

**CSF Files with NFL measures**

* General goal :
    - A master dataset of CSF NFL measure from all the available labs
* Main fields to be collected
    - RID, VISCODE (or VISCODE2)
    - CSF_NFL
    - Some timestamp indicating the sample collection time, depending on the lab

**Examine each lab file and the available fields**

In [ ]:
if debug:
    for k in dataframes["CSF"]["NFL"] :
        print('#lab ="{}"'.format(k))
        #print(k)

**Check the dictionary where fields are unclear**

In [ ]:
if debug:
    marker = "CSF"
    measure = "NFL"
    lab ="BLENNOWLAB_1_2014-11-13"
    print(lab)
    df = dataframes[marker][measure][lab]["df"]
    print(df.columns)
    print(df.info())
    dict_unc = dataframes[marker][measure][lab]["dict"]
    dict_unc[["FLDNAME", "TEXT"]]
    df.sample(5)

**Collect all CSF_NFL datasets in a standardized format**

In [ ]:
csf_nfl = {}

In [ ]:
csf_nfl_columns = ["RID", "VISCODE", "CSF_NFL", "update_stamp", "source"]

**BLENNOWLAB_1_2014-11-13**
 - Looks standard
 - Fields of interest 
  - CSFNFL

In [ ]:
marker = "CSF"
measure = "NFL"
lab ="BLENNOWLAB_1_2014-11-13"
df = dataframes[marker][measure][lab]["df"]
df = df[["RID", "VISCODE", "CSFNFL", "update_stamp"]]
df = df.dropna()
df["source"] = lab
csf_nfl[lab] = pd.DataFrame(df)
csf_nfl[lab].columns = csf_nfl_columns

In [ ]:
if debug: csf_nfl[lab].sample(5)

In [ ]:
if debug :
    print('--')
    print('dataframe dictionary for CSF NFL {}: csf_nfl')
    print('with columns : ', csf_nfl_columns)

<a id="csf-all" />

**CSF Files with Unspecified(ALL) measures**

* General goal :
    - A master dataset of CSF files where measures are unspecified (likely all of ABETA, TAU, PTAU, NFL)
* Main fields to be collected
    - RID, VISCODE (or VISCODE2)
    - where available : CSF_AB42, CSF_AB40, CSF_AB4240, CSF_TAU, CSF_PTAU
    - Some timestamp indicating the sample collection time, depending on the lab

Examine each lab file and the available fields

In [ ]:
if debug:
    for k in dataframes["CSF"]["ALL"] :
        print('#lab ="{}"'.format(k))
        #print(k)

**Collect all CSF_ALL datasets in a standardized format**

In [ ]:
csf_all = {}

In [ ]:
csf_all_columns = ["RID", "VISCODE", "CSF_AB42", "CSF_AB40", "CSF_AB4240", "CSF_TAU", "CSF_PTAU", "update_stamp", "source"]

**Check the dictionary where fields are unclear**

In [ ]:
if debug:
    marker = "CSF"
    measure = "ALL"
    #lab ="UPENN_ELECSYS_1_GO_2_3_2021-01-04"
    #lab ="UPENN_ELECSYS_1_GO_2_2017-04-19"
    #lab ="UPENN_ELECSYS_3_Bt1_2019-07-29"
    #lab ="UPENN_DIAN_1_GO_2_2018-04-09"
    #lab ="LOCALLAB_1_GO_2_3_2022-01-24"
    lab ="UPENN_MASTER_1_GO_2_2018-04-09"
    print(lab)
    df = dataframes[marker][measure][lab]["df"]
    print(df.columns)
    print(df.info())
    dict_unc = dataframes[marker][measure][lab]["dict"]
    dict_unc[["FLDNAME", "TEXT"]]
    df.sample(5)

**UPENN_MASTER_1_GO_2_2018-04-09**
 - Contains ~5800 entries, with a field 'BATCH' with values ['UPENNBIOMK', 'UPENNBIOMK2', 'MEDIAN', ...,'UPENNBIOMK7', 'UPENNBIOMK8']
 - Does not contain AB40
 - ABETA, TAU, PTAU fields are scaled to 'UPENNBIOMK', also extra '_RAW' fields for each of them
 - Fields of interest 
  - 'ABETA'
  - 'PTAU'
  - 'TAU'

In [ ]:
marker = "CSF"
measure = "ALL"
lab ="UPENN_MASTER_1_GO_2_2018-04-09"
df = dataframes[marker][measure][lab]["df"]
df = df[["RID", "VISCODE", "ABETA", "TAU", "PTAU", "update_stamp"]]
df = df.dropna()
df["AB40"] = np.NaN
df["AB4240"] = np.NaN
df["source"] = lab
csf_all[lab] = pd.DataFrame(df[["RID", "VISCODE", "ABETA", "AB40", "AB4240", "TAU", "PTAU", "update_stamp", "source"]])
csf_all[lab].columns = csf_all_columns

In [ ]:
if debug: csf_all[lab].sample(5)

**UPENN_DIAN_1_GO_2_2018-04-09**
 - Contains - AB42, AB40, AB4240, PTAU181, TAU
 - Fields of interest 
  - 'AB40'
  - 'ABETA'
  - 'PTAU'
  - 'TAU'
  - 'A4240'

In [ ]:
marker = "CSF"
measure = "ALL"
lab ="UPENN_DIAN_1_GO_2_2018-04-09"
df = dataframes[marker][measure][lab]["df"]
df = df[["RID", "VISCODE2", "ABETA", "AB40", "A4240", "TAU", "PTAU", "update_stamp"]]
#df = df.dropna() - 1 record has non-null ABeta but null tau 
df["source"] = lab
csf_all[lab] = pd.DataFrame(df)
csf_all[lab].columns = csf_all_columns

In [ ]:
if debug: csf_all[lab].info()

**UPENN_ELECSYS_3_Bt1_2019-07-29**
 - Contains - 'ABETA40', 'ABETA42', 'PTAU', 'TAU'
 - AB4240 ratio to be calculated

In [ ]:
marker = "CSF"
measure = "ALL"
lab ="UPENN_ELECSYS_3_Bt1_2019-07-29"
df = dataframes[marker][measure][lab]["df"]
df = df[["RID", "VISCODE2", "ABETA42", "ABETA40", "TAU", "PTAU", "update_stamp"]]
df["AB4240"] = df["ABETA42"]/df["ABETA40"]
#df = df.dropna() - Some records have non-null ABeta but null tau 
df["source"] = lab
csf_all[lab] = pd.DataFrame(df[["RID", "VISCODE2", "ABETA42", "ABETA40", "AB4240", "TAU", "PTAU", "update_stamp", "source"]])
csf_all[lab].columns = csf_all_columns

In [ ]:
if debug: csf_all[lab].sample(5)

**UPENN_ELECSYS_1_GO_2_2017-04-19**
 - Contains ~2400 entries (compared to 216 in UPENN_ELECSYS_1_GO_2_3 file)
 - Does not contain AB40
 - Fields of interest 
  - 'ABETA' - has values like '>1700'
  - 'PTAU'
  - 'TAU'

In [ ]:
marker = "CSF"
measure = "ALL"
lab ="UPENN_ELECSYS_1_GO_2_2017-04-19"
df = dataframes[marker][measure][lab]["df"]
df = df[["RID", "VISCODE2", "ABETA", "TAU", "PTAU", "update_stamp"]]
df = df.dropna()
df["AB40"] = np.NaN
df["AB4240"] = np.NaN
df["source"] = lab
csf_all[lab] = pd.DataFrame(df[["RID", "VISCODE2", "ABETA", "AB40", "AB4240", "TAU", "PTAU", "update_stamp", "source"]])
csf_all[lab].columns = csf_all_columns

In [ ]:
if debug: csf_all[lab].sample(5)

**UPENN_ELECSYS_1_GO_2_3_2021-01-04**
 - Contains - AB42, AB40, AB4240, PTAU181, TAU
 - Fields of interest 
  - 'AB40'
  - 'ABETA'
  - 'PTAU'
  - 'TAU'
  - 'AB4240'

In [ ]:
marker = "CSF"
measure = "ALL"
lab ="UPENN_ELECSYS_1_GO_2_3_2021-01-04"
df = dataframes[marker][measure][lab]["df"]
df = df[["RID", "VISCODE2", "ABETA", "AB40", "AB4240", "TAU", "PTAU", "EXAMDATE"]]
#df = df.dropna() - Some records have non-null ABeta but null tau 
df["source"] = lab
csf_all[lab] = pd.DataFrame(df)
csf_all[lab].columns = csf_all_columns

In [ ]:
if debug: csf_all[lab].sample(5)

In [ ]:
if debug :
    print('--')
    print('dataframe dictionary for CSF ALL {}: csf_all')
    print('with columns : ', csf_all_columns)

<a id="csf-questions" />

#### CSF Questions 
* CSF
    - NFL 
        * One file available, BLENNOWLAB ADNI 1, CSF_NFL field, 415 entries
    - AB42/AB40
        * Two files available : AB42, AB40, AB4240 available in both
             - FUJIREBIO ADNI 1,GO,2 - 440 entries
             - EUROIMMUN ADNI 1,GO,2 - VISCODE2 **240** non-null but BETA_AMYLOID_42_40 **278 non-null** ?
    - General : Files with all or some of [ABETA, TAU and PTAU]
        * 5 files available : TAU, PTAU, ABETA42 in all, ABETA40 and AB4240 in few
            - UPENN ELECSYS ADNI 1_GO_2_3 : All the fields available, ~214 entries
            - UPENN ELECSYS ADNI 1_GO_2 : ~2400 entries, but no AB40 (nor the ratio)
            - UPENN ELECSYS ADNI 3, Batch 1 : ~497 entries, ratio to be calculated
            - UPENN ADNI DIAN ADNI 1_GO_2 : All the fields available, ~422 entries
            - UPENN MASTER ADNI 1_GO_2 : 
                * Contains ~5800 entries, no AB40 nor the ratio
                * with a field 'BATCH' with values ['UPENNBIOMK', 'UPENNBIOMK2', 'MEDIAN', ...,'UPENNBIOMK7', 'UPENNBIOMK8']
                * ABETA, TAU, PTAU fields are scaled to 'UPENNBIOMK', also extra '_RAW' fields for each of them


<a id="plasma" />

### PLASMA

**Available PLASMA Measures**

In [ ]:
if debug:
    for k in dataframes["PLASMA"] :
        print(k)

<a id="plasma-abeta" />

**PLASMA Files with ABETA measures**

* General goal :
    - A master dataset of ABETA measure from all the available labs
* Main fields to be collected
    - RID, VISCODE (or VISCODE2)
    - ABETA1-42
    - ABETA1-40
    - Ratio ABETA42/40
    - Some timestamp indicating the sample collection time, depending on the lab

**Examine each lab file for the required fields**

In [ ]:
if debug:
    for k in dataframes["PLASMA"]["ABETA"] :
        #print('#lab ="{}"'.format(k))
        print(k)

<a id="plasma-abeta-exploration" />

**Check the dictionary where fields are unclear**

In [ ]:
if debug:
    #lab ="ARACLON_1_GO_2_2016-02-16"
    #lab ="FNIH_UWASH_1_GO_2_2021-11-05"
    #lab ="UPENN_1_"
    #lab ="FNIH_SHIMADZU_1_GO_2_2021-05-25"
    #lab ="BATEMAN_2019-06-21"
    #lab ="FNIH_VUMC_1_GO_2_2021-05-25"
    #lab ="FNIH_QUANTERIX_1_GO_2_2021-05-25"
    #lab ="FNIH_ROCHE_1_GO_2_2021-05-25"
    lab ="FNIH_GOTHENBURG_1_GO_2_2021-05-25"
    print(lab)
    df = dataframes["PLASMA"]["ABETA"][lab]["df"]
    print(df.columns)
    dict_unc = dataframes["PLASMA"]["ABETA"][lab]["dict"]
    dict_unc[["FLDNAME", "TEXT"]]
    #df.info()

**Collect all PLASMA_ABETA datasets in a standardized format**

In [ ]:
plasma_abeta = {}

**Collect the following fields from all the groups**

In [ ]:
plasma_abeta_columns = ["RID", "VISCODE", "PLASMA_AB42", "PLASMA_AB40", "PLASMA_AB4240", "update_stamp", "source"]

**FNIH_GOTHENBURG_1_GO_2_2021-05-25**
 - Looks standard, ratio to be calculated
 - Fields of interest 
  - AB_1_42
  - AB_1_40
 - NULL values with '-', marked as technical issues

In [ ]:
lab ="FNIH_GOTHENBURG_1_GO_2_2021-05-25"
df = dataframes["PLASMA"]["ABETA"][lab]["df"]
df = df[["RID", "VISCODE2", "AB_1_42", "AB_1_40", "update_stamp"]]
df = df.replace('-', np.NaN)
df = df.dropna()
df.AB_1_42 = df.AB_1_42.astype('float')
df.AB_1_40 = df.AB_1_40.astype('float')
df["AB4240"] = df["AB_1_42"]/df["AB_1_40"]
df["source"] = lab
plasma_abeta[lab] = pd.DataFrame(df[["RID", "VISCODE2", "AB_1_42", "AB_1_40", "AB4240", "update_stamp", "source"]])
plasma_abeta[lab].columns = plasma_abeta_columns

In [ ]:
if debug: plasma_abeta[lab].sample(5)

**FNIH_ROCHE_1_GO_2_2021-05-25**
 - Data is condensed - column ASSAY has 'Abeta1-42' or 'Abeta1-40' and RESULT has the corresponding value
 - update_stamp is the same for all of them

In [ ]:
lab ="FNIH_ROCHE_1_GO_2_2021-05-25"
df = dataframes["PLASMA"]["ABETA"][lab]["df"]
df = df[["RID", "VISCODE2", "ASSAY", "RESULT", "update_stamp"]]
df = df.pivot(index=["RID","VISCODE2","update_stamp"], columns='ASSAY')["RESULT"].reset_index()
df = df.dropna()
df["AB4240"] = df["Abeta1-42"]/df["Abeta1-40"]
df["source"] = lab
plasma_abeta[lab] = pd.DataFrame(df[["RID", "VISCODE2", "Abeta1-42", "Abeta1-40", "AB4240", "update_stamp", "source"]])
plasma_abeta[lab].columns = plasma_abeta_columns

In [ ]:
if debug: plasma_abeta[lab].sample(5)

**FNIH_QUANTERIX_1_GO_2_2021-05-25**
 - Data has replicate entries
 - There are values w.r.t a DILUTION_FACTOR = 4
 - Fields of Interest :
     - 'AB1_42_AVE_CONC' or 'AB1_42_DILUTION_CORRECTED_CONC'
     - 'AB1_40_AVE_CONC' or 'AB1_40_DILUTION_CORRECTED_CONC'
 - Going with AB1_40_DILUTION_CORRECTED_CONC for now

In [ ]:
lab ="FNIH_QUANTERIX_1_GO_2_2021-05-25"
df = dataframes["PLASMA"]["ABETA"][lab]["df"]
df = df[["RID", "VISCODE2", "AB1_42_DILUTION_CORRECTED_CONC", "AB1_40_DILUTION_CORRECTED_CONC", "update_stamp"]]
df = df.dropna()
df["AB4240"] = df["AB1_42_DILUTION_CORRECTED_CONC"]/df["AB1_40_DILUTION_CORRECTED_CONC"]
df["source"] = lab
plasma_abeta[lab] = pd.DataFrame(df[["RID", "VISCODE2", "AB1_42_DILUTION_CORRECTED_CONC", "AB1_40_DILUTION_CORRECTED_CONC", "AB4240", "update_stamp", "source"]])
plasma_abeta[lab].columns = plasma_abeta_columns

In [ ]:
if debug: plasma_abeta[lab].sample(5)

**FNIH_VUMC_1_GO_2_2021-05-25**
 - Straighforward, ratio to be calculated
 - Fields of Interest :
     - ABETA42
     - ABETA40
 - Some comments on ABETA40 in the Dictionary

In [ ]:
lab ="FNIH_VUMC_1_GO_2_2021-05-25"
df = dataframes["PLASMA"]["ABETA"][lab]["df"]
df = df[["RID", "VISCODE2", "ABETA42", "ABETA40", "update_stamp"]]
df = df.dropna()
df["AB4240"] = df["ABETA42"]/df["ABETA40"]
df["source"] = lab
plasma_abeta[lab] = pd.DataFrame(df[["RID", "VISCODE2", "ABETA42", "ABETA40", "AB4240", "update_stamp", "source"]])
plasma_abeta[lab].columns = plasma_abeta_columns

In [ ]:
if debug: plasma_abeta[lab].sample(5)

**BATEMAN_2019-06-21**
 - Many values, picking N15 ones
 - Fields of Interest : (units of AB42 and AB40 different from others)
  - PEAK_AREA_ABETA40_N15_TOUSE
  - PEAK_AREA_ABETA42_N15_TOUSE
  - RATIO_ABETA42_40_BY_ISTD_TOUSE

In [ ]:
lab ="BATEMAN_2019-06-21"
df = dataframes["PLASMA"]["ABETA"][lab]["df"]
df = df[["RID", "VISCODE2", "PEAK_AREA_ABETA42_N15_TOUSE", "PEAK_AREA_ABETA40_N15_TOUSE", 
         "RATIO_ABETA42_40_BY_ISTD_TOUSE", "update_stamp"]]
df = df.dropna()
df["source"] = lab
plasma_abeta[lab] = pd.DataFrame(df)
plasma_abeta[lab].columns = plasma_abeta_columns

In [ ]:
lab ="BATEMAN_2022-11-18"
df = dataframes["PLASMA"]["ABETA"][lab]["df"]
df = df[["RID", "VISCODE2", "Abeta_42_conc", "Abeta_40_conc", 
         "Abeta_4240_Standardized", "update_stamp"]]
df = df.dropna()
df["source"] = lab
plasma_abeta[lab] = pd.DataFrame(df)
plasma_abeta[lab].columns = plasma_abeta_columns

In [ ]:
if debug: plasma_abeta[lab].sample(5)

**FNIH_SHIMADZU_1_GO_2_2021-05-25**
 - COMPOSITE_BIOMARKER	: combining normalized scores of APP669711/AB 142 and AB 140/AB 142 with a weight of 1:1. 

In [ ]:
lab ="FNIH_SHIMADZU_1_GO_2_2021-05-25"
df = dataframes["PLASMA"]["ABETA"][lab]["df"]
df = df[["RID", "VISCODE2", "AB1_42", "AB1_40", "update_stamp"]]
df = df.dropna()
df["AB4240"] = df["AB1_42"]/df["AB1_40"]
df["source"] = lab
plasma_abeta[lab] = pd.DataFrame(df[["RID", "VISCODE2", "AB1_40", "AB1_40", "AB4240", "update_stamp", "source"]])
plasma_abeta[lab].columns = plasma_abeta_columns

In [ ]:
if debug : plasma_abeta[lab].sample(5)

**UPENN_1_**
 - Straightforward fields, some blank values
 - downloaded file : "UPENN - Plasma Biomarker Data [ADNI1] " - no upload date

In [ ]:
lab ="UPENN_1_"
df = dataframes["PLASMA"]["ABETA"][lab]["df"]
df = df.dropna()
df["AB4240"] = df["AB42"]/df["AB40"]
df["source"] = lab
plasma_abeta[lab] = pd.DataFrame(df[["RID", "VISCODE", "AB42", "AB40", "AB4240", "update_stamp", "source"]])
plasma_abeta[lab].columns = plasma_abeta_columns

In [ ]:
if debug: plasma_abeta[lab].sample(5)

**FNIH_UWASH_1_GO_2_2021-11-05**
 - Main fields (specified in TEXT in Dictionary) 
  - STANDARDIZED_PLASMAAB42
  - STANDARDIZED_PLASMAAB40
  - STANDARDIZED_PLASMAAB4240
 - 1 record with the field "QC_FLAGS" = "(00) No signal" has blank values for the above fields

In [ ]:
lab ="FNIH_UWASH_1_GO_2_2021-11-05"
df = dataframes["PLASMA"]["ABETA"][lab]["df"]
df = df[~(~df.QC_FLAGS.isna() & df.QC_FLAGS.str.contains('No signal'))]
df = df[["RID", "VISCODE2", "STANDARDIZED_PLASMAAB42", "STANDARDIZED_PLASMAAB40"
                                              ,"STANDARDIZED_PLASMAAB4240", "update_stamp"]]
df = df.dropna()
df["source"] = lab
plasma_abeta[lab] = pd.DataFrame(df)
plasma_abeta[lab].columns = plasma_abeta_columns

In [ ]:
if debug: plasma_abeta[lab].sample(5)

**ARACLON_1_GO_2_2016-02-16**

In [ ]:
lab ="ARACLON_1_GO_2_2016-02-16"
df = dataframes["PLASMA"]["ABETA"][lab]["df"].drop(['FP40','FP42'], axis=1)
df = df.replace('ND', np.NaN)
df = df.replace('nd', np.NaN)
df = df.replace('>ULOQ', np.NaN)
df = df.dropna()
df.TP42 = df.TP42.astype('float')
df.TP40 = df.TP40.astype('float')
df["AB4240"] = df["TP42"]/df["TP40"]
df["source"] = lab
plasma_abeta[lab] = pd.DataFrame(df[["RID", "VISCODE", "TP42", "TP40", "AB4240", "update_stamp", "source"]])
plasma_abeta[lab].columns = plasma_abeta_columns

In [ ]:
if debug: plasma_abeta[lab].sample(5)

In [ ]:
print('--')
print('dataframe dictionary for PLASMA ABETA {}: plasma_abeta')
print('with columns : ', plasma_abeta_columns)

<a id="plasma-tau" />

**PLASMA Files with TAU measures**

* General goal :
    - A master dataset of PLASMA TAU measure from all the available labs
* Main fields to be collected
    - RID, VISCODE (or VISCODE2)
    - PLASMATAU
    - Some timestamp indicating the sample collection time, depending on the lab

**Collect the following fields from all the groups**

In [ ]:
plasma_tau_columns = ["RID", "VISCODE", "PLASMA_TAU", "update_stamp", "source"]

In [ ]:
if debug:
    for k in dataframes["PLASMA"]["TAU"] :
        #print('#lab ="{}"'.format(k))
        print(k)

**Collect all PLASMA_TAU datasets in a standardized format**

In [ ]:
plasma_tau = {}

**BLENNOWLAB_1_2015-08-04**

In [ ]:
lab = "BLENNOWLAB_1_2015-08-04"
df = dataframes["PLASMA"]["TAU"][lab]["df"][["RID", "VISCODE", "PLASMATAU", "update_stamp"]]
df = df.dropna()
df["source"] = lab
plasma_tau[lab] = pd.DataFrame(df)
plasma_tau[lab].columns = plasma_tau_columns

In [ ]:
if debug: plasma_tau[lab].sample(5)

In [ ]:
if debug:
    print('--')
    print('dataframe dictionary for PLASMA TAU {}: plasma_tau')
    print('with columns : ', plasma_tau_columns)

<a id="plasma-ptau" />

**PLASMA Files with P-TAU181 measures**

* General goal :
    - A master dataset of PLASMA P-TAU181 measure from all the available labs
* Main fields to be collected
    - RID, VISCODE (or VISCODE2)
    - PLASMAPTAU181
    - Some timestamp indicating the sample collection time, depending on the lab

**Collect the following fields from all the groups**

In [ ]:
plasma_ptau_columns = ["RID", "VISCODE", "PLASMA_PTAU", "update_stamp", "source"]

In [ ]:
if debug:
    for k in dataframes["PLASMA"]["PTAU"] :
        #print('#lab ="{}"'.format(k))
        print(k)

**Collect all PLASMA_PTAU181 datasets in a standardized format**

In [ ]:
plasma_ptau = {}

**GOTHENBURG_1_GO_2_2020-06-18**

In [ ]:
measure = "PTAU"
lab = "GOTHENBURG_1_GO_2_2020-06-18"
df = dataframes["PLASMA"][measure][lab]["df"][["RID", "VISCODE2", "PLASMAPTAU181", "update_stamp"]]
df = df.dropna()
df["source"] = lab
plasma_ptau[lab] = pd.DataFrame(df)
plasma_ptau[lab].columns = plasma_ptau_columns

In [ ]:
if debug: plasma_ptau[lab].sample(5)

In [ ]:
if debug :
    print('--')
    print('dataframe dictionary for PLASMA PTAU181 {}: plasma_ptau')
    print('with columns : ', plasma_ptau_columns)

<a id="plasma-nfl" />

**PLASMA Files with Neurofilament Light(NFL) measures**

* General goal :
    - A master dataset of PLASMA NFL measure from all the available labs
* Main fields to be collected
    - RID, VISCODE (or VISCODE2)
    - PLASMA_NFL
    - Some timestamp indicating the sample collection time, depending on the lab

**Collect the following fields from all the groups**

In [ ]:
plasma_nfl_columns = ["RID", "VISCODE", "PLASMA_NFL", "update_stamp", "source"]

In [ ]:
if debug:
    for k in dataframes["PLASMA"]["NFL"] :
        print('#lab ="{}"'.format(k))
        #print(k)

**Collect all PLASMA_NFL datasets in a standardized format**

In [ ]:
plasma_nfl = {}

In [ ]:
if debug:
    lab ="BLENNOWLAB_ADNI_1_2018-10-03"
    lab ="BLENNOWLAB_1_GO_2_2018-10-03"
    print(lab)
    df = dataframes["PLASMA"]["NFL"][lab]["df"]
    df
    #print(df.columns)
    #dict_unc = dataframes["PLASMA"]["NFL"][lab]["dict"]
    #dict_unc[["FLDNAME", "TEXT"]]
    #df.info()

**BLENNOWLAB_ADNI_1_2018-10-03**

In [ ]:
measure = "NFL"
lab = "BLENNOWLAB_ADNI_1_2018-10-03"
df = dataframes["PLASMA"][measure][lab]["df"][["RID", "VISCODE2", "PLASMA_NFL", "update_stamp"]]
df = df.dropna()
df["source"] = lab
plasma_nfl[lab] = pd.DataFrame(df)
plasma_nfl[lab].columns = plasma_nfl_columns

In [ ]:
if debug: plasma_nfl[lab].sample(5)

**BLENNOWLAB_LONG_1_GO_2_2018-10-03**

In [ ]:
measure = "NFL"
lab = "BLENNOWLAB_1_GO_2_2018-10-03"
df = dataframes["PLASMA"][measure][lab]["df"][["RID", "VISCODE2", "PLASMA_NFL", "update_stamp"]]
df = df.dropna()
df["source"] = lab
plasma_nfl[lab] = pd.DataFrame(df)
plasma_nfl[lab].columns = plasma_nfl_columns

In [ ]:
if debug :
    print('--')
    print('dataframe dictionary for PLASMA NFL {}: plasma_nfl')
    print('with columns : ', plasma_nfl_columns)

In [ ]:
plasma_nfl_all = pd.concat([plasma_nfl[lab] for lab in plasma_nfl])
plasma_nfl_all = plasma_nfl_all[plasma_nfl_all.VISCODE != 'nv']
pnfl_all = plasma_nfl_all.groupby(['RID', 'VISCODE']).size().reset_index(name='counts')
plasma_nfl_all_dupes = plasma_nfl_all[plasma_nfl_all.duplicated(['RID','VISCODE'], keep=False)]
pnfl_all_sources = plasma_nfl_all_dupes.groupby(['RID', 'VISCODE'])['source'].apply(lambda x: ','.join(x)).reset_index()
#pab_all
#pnfl_all[pnfl_all['counts'] > 1]
#['source'].apply(lambda x: ','.join(x)).reset_index()
#pnfl_all_sources

In [ ]:
dfb1 = plasma_nfl["BLENNOWLAB_ADNI_1_2018-10-03"].copy()
dfb2 = plasma_nfl["BLENNOWLAB_1_GO_2_2018-10-03"].copy()
dfb1 = dfb1.reset_index(drop=True)
#dfb1.merge(dfb2, how='inner', on=['RID', 'VISCODE'])

<a id="plasma-questions" />

#### PLASMA Questions 
* PLASMA
    - NFL 
         * Two available files from BLENNOWLAB :
         * 1. Blennow Lab ADNI1 Plasma neurofilament light (NFL) [ADNI1] 
         * 2. Blennow Lab ADNI1-2 Plasma neurofilament light (NFL) longitudinal [ADNI1,GO,2] 
         * Both have non-overlapping RID+VISCODE combinations
         * File 2 mentions "Longitudinal" and has 53 duplicate entries (each RID+VISCODE combo twice, each on the same day differing in their DRAW_TIME and PLASMA_NFL value).
         * Average value for these cases?
    - AB42/AB40
         * ARACLON
          - Difference between "Free ABeta40 in plasma" and "Total ABeta40 in plasma" (same with ABeta42)
          - Some values are "ND" or ">ULOQ", mostly 42, one 40
         * FNIH_UWASH
          - Specified in their Dictionary : Use STANDARDIZED_PLASMAAB4240
         * UPENN_1_
          - "UPENN - Plasma Biomarker Data [ADNI1] " - no upload date
          - AB42 and AB40 units are different from others
         * FNIH_SHIMADZU
          - New field : COMPOSITE_BIOMARKER (APP669711/AB 142 and AB 140/AB 142)
          - Take 'AB1_42' and 'AB1_40' and calculate ratio?
         * BATEMAN Lab
          - Both AB42 qnd AB40 : Values for N14 and N15 ? (for e.g: PEAK_AREA_ABETA40_N14_TOUSE	vs PEAK_AREA_ABETA40_N15_TOUSE)
          - Mentioned : endogenous A40 (N14) and 15N labeled A40 internal standard (N15)
          - Choose N15 values?
          - Some records with QC_FLAGS = '00 No Ab signal;' or QC_STATUS = 'FAIL' still have non-zero ratio AB42/40. Is that normal?
         * FNIH_QUANTERIX
          - Two probable fields (for each of AB42 and AB40). Check the following three fields
            * AB1_40_AVE_CONC	AB1-40 average of the calibrated concentrations
            * AB1_40_DILUTION_FACTOR	AB1-40 dilution factor. Plasma samples were diluted 4-fold.
            * AB1_40_DILUTION_CORRECTED_CONC	Dilution corrected AB1-40 concentration of analyte in pg/mL 
          - Prefer AB1_40_DILUTION_CORRECTED_CONC ?

 - RID
 - VISCODE = bl, m06 etc.. if any dataset has 'VISCODE2', use that
     sc = bl
     f = failed
 - Ignore PTID
 - DX_bl : Diagnosis Baseline
 - DX 
 - CSF - 
 - Plasma Abeta42/Abeta40 Bateman
 - Gothenburg - P-tau, Abeta
 - NFL
 - MRI - some in ADNIMERGE
 - PET : AV1451 - tau
         AV45 - Abeta
          - FDG, measure for glucose metabolism
          
          
#### For each of the individual datasets, check RID+VISCODE combo, and verify if there are repetitions and why

* 15748 --> ADNIMERGE
* PET :
        Abeta : 2998 --> PET_AV45_UCB-1_GO_2_3
        Tau : 1304 --> PET_AV1451_UCB-1_GO_2_3 -> Sounds right
* CSF :
        ABETA, PTAU, TAU, AB42/40: 216 --> CSF_UPENN_ELECSYS-1_GO_2_3  -> XXXX not complete
* PLASMA :
        PTAU : 3758 --> PLASMA_GOTHENBURG-1_GO_2 -> Sounds right
        ABETA42/40 : 622 --> PLASMA_BATEMAN -> XXXX not what we're looking for. 
        NFL : 3762 --> PLASMA_NFL_BLENNOWLAB-1_GO_2 -> Sounds right
* MRI :
       3570 --> MRI_FREESURF_UCSF_1
       3311 --> MRI_FREESURF_UCSF_GO_2 -> Sounds right

In [ ]:
def incidentDem() :
    dfa = ADNIMERGE[['RID', 'Month', 'DX_bl', 'DX']]

    dfa['Incident_Dem'] = (~dfa['DX_bl'].str.contains('AD', na=False)) & dfa['DX'].str.contains('Dement', na=False)

    dfa = dfa.fillna('-')
    dfa = dfa.sort_values(by=['RID', 'Month'])

    dfa.Incident_Dem = dfa.Incident_Dem.astype('int')
    dfa.Incident_Dem = dfa.Incident_Dem.astype('string')
    dfa.DX_bl = dfa.DX_bl.astype('string')
    dfa.DX = dfa.DX.astype('string')
    dfa.Month = dfa.Month.astype('string')

    newdf = dfa.groupby(['RID']).agg(lambda x: ','.join(x))

    return newdf